# Has Coverage Merge
## Merging Meters Coverage and Route Coverage Datasets

This notebook performs a left join on ASSETNUM to combine meter coverage data with route coverage data.

### Step 1: Import Libraries

In [1]:
import pandas as pd
import pickle
import os

print("Libraries imported successfully!")

Libraries imported successfully!


### Step 2: Load Meters Coverage Data

In [2]:
# Load meters coverage data
with open('meters-coverage.pkl', 'rb') as f:
    meters_df = pickle.load(f)

print(f"Meters Coverage Shape: {meters_df.shape}")
print(f"\nColumns: {meters_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
meters_df.head()

Meters Coverage Shape: (8223, 8)

Columns: ['ASSETNUM', 'METER_COUNT', 'MAX_LASTREADING_DATE', 'ASSET_DESCRIPTION', 'DEPT', 'CLASS', 'READING_WITHIN_1YR', 'HAS_GM']

First 5 rows:


,ASSETNUM,METER_COUNT,MAX_LASTREADING_DATE,ASSET_DESCRIPTION,DEPT,CLASS,READING_WITHIN_1YR,HAS_GM
0,A100001,83,NaT,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,N,N
1,A100002,91,2026-01-20 20:27:09.260,SMURF REAR INSTALL ROBOT END OF ARM TOOL - ODY...,Head Assembly,1AF,Y,Y
2,A100003,84,2024-05-10 17:11:50.072,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - RI...,Head Assembly,1AF,N,N
3,A100004,88,2026-01-20 20:28:24.798,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - OD...,Head Assembly,1AF,Y,Y
4,A100005,84,2025-12-05 01:01:06.922,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,Y


### Step 3: Load Route Coverage Data

In [3]:
# Load route coverage data
with open('route-coverage.pkl', 'rb') as f:
    routes_df = pickle.load(f)

print(f"Route Coverage Shape: {routes_df.shape}")
print(f"\nColumns: {routes_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
routes_df.head()

Route Coverage Shape: (1466, 10)

Columns: ['ASSETNUM', 'ASSET_DESC', 'CLASS', 'DEPT', 'IR_HAS', 'UL_HAS', 'VI_HAS', 'LU_HAS', 'MC_HAS', 'ZD_HAS']

First 5 rows:


,ASSETNUM,ASSET_DESC,CLASS,DEPT,IR_HAS,UL_HAS,VI_HAS,LU_HAS,MC_HAS,ZD_HAS
0,A100001,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,Y,N,N,N,N,N
1,A100005,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,N,N,N,N,N
2,A100006,KNUCKLE ISLAND BEARING PRESS #1,Press,1AF,Y,N,N,N,N,N
3,A100024,"RSAT, ELECTRICAL PANEL, HMI 1, CONVEYOR LOAD",Panel,1AF,Y,N,N,N,N,N
4,A100042,SMURF ENGINE SUSPENSION INSTALL ROBOT FANUC MA...,Manipulator,1AF,N,N,N,Y,N,N


### Step 4: Examine Data Before Merge

In [4]:
print("METERS-COVERAGE:")
print(f"  Total records: {len(meters_df)}")
print(f"  Unique ASSETNUM: {meters_df['ASSETNUM'].nunique()}")
print(f"  Data types:\n{meters_df.dtypes}")

print("\n" + "="*80 + "\n")

print("ROUTE-COVERAGE:")
print(f"  Total records: {len(routes_df)}")
print(f"  Unique ASSETNUM: {routes_df['ASSETNUM'].nunique()}")
print(f"  Data types:\n{routes_df.dtypes}")

METERS-COVERAGE:
  Total records: 8223
  Unique ASSETNUM: 8223
  Data types:
ASSETNUM                      category
METER_COUNT                      int64
MAX_LASTREADING_DATE    datetime64[ns]
ASSET_DESCRIPTION               object
DEPT                            object
CLASS                           object
READING_WITHIN_1YR              object
HAS_GM                          object
dtype: object


ROUTE-COVERAGE:
  Total records: 1466
  Unique ASSETNUM: 1466
  Data types:
ASSETNUM      category
ASSET_DESC      object
CLASS         category
DEPT            object
IR_HAS          object
UL_HAS          object
VI_HAS          object
LU_HAS          object
MC_HAS          object
ZD_HAS          object
dtype: object


### Step 5: Check for Overlapping Columns

In [5]:
# Check for overlapping columns (excluding the join key)
meters_cols = set(meters_df.columns)
routes_cols = set(routes_df.columns)

overlapping = (meters_cols & routes_cols) - {'ASSETNUM'}

print(f"Overlapping columns (excluding ASSETNUM): {overlapping}")
print(f"\nThese will get suffixes after the merge.")

Overlapping columns (excluding ASSETNUM): {'CLASS', 'DEPT'}

These will get suffixes after the merge.


### Step 6: Perform Left Join on ASSETNUM

In [6]:
# Perform left join
merged_df = meters_df.merge(
    routes_df,
    on='ASSETNUM',
    how='left',
    suffixes=('_METERS', '_ROUTES')
)

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nColumns in merged dataset:")
for i, col in enumerate(merged_df.columns, 1):
    print(f"  {i:2d}. {col}")

Merged dataset shape: (8223, 17)

Columns in merged dataset:
   1. ASSETNUM
   2. METER_COUNT
   3. MAX_LASTREADING_DATE
   4. ASSET_DESCRIPTION
   5. DEPT_METERS
   6. CLASS_METERS
   7. READING_WITHIN_1YR
   8. HAS_GM
   9. ASSET_DESC
  10. CLASS_ROUTES
  11. DEPT_ROUTES
  12. IR_HAS
  13. UL_HAS
  14. VI_HAS
  15. LU_HAS
  16. MC_HAS
  17. ZD_HAS


### Step 7: Examine Merged Data

In [7]:
# Display first 10 rows
print("First 10 rows of merged data:")
merged_df.head(10)

First 10 rows of merged data:


,ASSETNUM,METER_COUNT,MAX_LASTREADING_DATE,ASSET_DESCRIPTION,DEPT_METERS,CLASS_METERS,READING_WITHIN_1YR,HAS_GM,ASSET_DESC,CLASS_ROUTES,DEPT_ROUTES,IR_HAS,UL_HAS,VI_HAS,LU_HAS,MC_HAS,ZD_HAS
0,A100001,83,NaT,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,N,N,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,Y,N,N,N,N,N
1,A100002,91,2026-01-20 20:27:09.260,SMURF REAR INSTALL ROBOT END OF ARM TOOL - ODY...,Head Assembly,1AF,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A100003,84,2024-05-10 17:11:50.072,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - RI...,Head Assembly,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A100004,88,2026-01-20 20:28:24.798,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - OD...,Head Assembly,1AF,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A100005,84,2025-12-05 01:01:06.922,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,Y,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,N,N,N,N,N
5,A100006,84,2024-05-23 12:16:17.124,KNUCKLE ISLAND BEARING PRESS #1,Press,1AF,N,N,KNUCKLE ISLAND BEARING PRESS #1,Press,1AF,Y,N,N,N,N,N
6,A100007,1,NaT,REAR KNUCKLE AUTO TIGHTEN (PARENT),Power Tool,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A100008,1,NaT,"INLINE BURNISHING MACHINE (PARENT), BURNISHER",Grinder,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A100009,1,NaT,REAR SUSPENSION CONVEYOR (PARENT),Power Tool,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A100010,1,NaT,REAR SUSPENSION AUTO TIGHTEN (PARENT),Power Tool,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Check data types
print("Data types in merged dataset:")
merged_df.dtypes

Data types in merged dataset:


ASSETNUM                        object
METER_COUNT                      int64
MAX_LASTREADING_DATE    datetime64[ns]
ASSET_DESCRIPTION               object
DEPT_METERS                     object
CLASS_METERS                    object
READING_WITHIN_1YR              object
HAS_GM                          object
ASSET_DESC                      object
CLASS_ROUTES                  category
DEPT_ROUTES                     object
IR_HAS                          object
UL_HAS                          object
VI_HAS                          object
LU_HAS                          object
MC_HAS                          object
ZD_HAS                          object
dtype: object

### Step 8: Analyze Merge Results

In [9]:
# Calculate match statistics
total_records = len(merged_df)
matched_records = merged_df['IR_HAS'].notna().sum()
match_rate = (matched_records / total_records * 100)

print("MERGE STATISTICS:")
print("="*60)
print(f"Total records in merged dataset: {total_records:,}")
print(f"Records with route coverage data: {matched_records:,} ({match_rate:.1f}%)")
print(f"Records without route coverage: {total_records - matched_records:,} ({100-match_rate:.1f}%)")

print("\n" + "="*60)
print("COVERAGE SUMMARY:")
print("="*60)

# GM Coverage (from meters)
gm_coverage = (merged_df['HAS_GM'] == 'Y').sum()
print(f"GM (General Metering) Coverage: {gm_coverage:,} assets ({gm_coverage/total_records*100:.1f}%)")

# Route-based coverage
if 'IR_HAS' in merged_df.columns:
    ir_coverage = (merged_df['IR_HAS'] == 'Y').sum()
    print(f"IR (Infrared) Coverage:         {ir_coverage:,} assets ({ir_coverage/total_records*100:.1f}%)")

if 'UL_HAS' in merged_df.columns:
    ul_coverage = (merged_df['UL_HAS'] == 'Y').sum()
    print(f"UL (Ultrasound) Coverage:        {ul_coverage:,} assets ({ul_coverage/total_records*100:.1f}%)")

if 'VI_HAS' in merged_df.columns:
    vi_coverage = (merged_df['VI_HAS'] == 'Y').sum()
    print(f"VI (Vibration) Coverage:         {vi_coverage:,} assets ({vi_coverage/total_records*100:.1f}%)")

if 'LU_HAS' in merged_df.columns:
    lu_coverage = (merged_df['LU_HAS'] == 'Y').sum()
    print(f"LU (Lubrication) Coverage:       {lu_coverage:,} assets ({lu_coverage/total_records*100:.1f}%)")

if 'MC_HAS' in merged_df.columns:
    mc_coverage = (merged_df['MC_HAS'] == 'Y').sum()
    print(f"MC Coverage:                     {mc_coverage:,} assets ({mc_coverage/total_records*100:.1f}%)")

if 'ZD_HAS' in merged_df.columns:
    zd_coverage = (merged_df['ZD_HAS'] == 'Y').sum()
    print(f"ZD Coverage:                     {zd_coverage:,} assets ({zd_coverage/total_records*100:.1f}%)")

MERGE STATISTICS:
Total records in merged dataset: 8,223
Records with route coverage data: 615 (7.5%)
Records without route coverage: 7,608 (92.5%)

COVERAGE SUMMARY:
GM (General Metering) Coverage: 5,005 assets (60.9%)
IR (Infrared) Coverage:         360 assets (4.4%)
UL (Ultrasound) Coverage:        207 assets (2.5%)
VI (Vibration) Coverage:         345 assets (4.2%)
LU (Lubrication) Coverage:       131 assets (1.6%)
MC Coverage:                     0 assets (0.0%)
ZD Coverage:                     86 assets (1.0%)


### Step 9: Sample Assets with and without Route Coverage

In [10]:
# Show sample of assets WITH route coverage
print("Sample assets WITH route coverage:")
with_coverage = merged_df[merged_df['IR_HAS'].notna()].head(5)
display(with_coverage)

print("\n" + "="*80 + "\n")

# Show sample of assets WITHOUT route coverage
print("Sample assets WITHOUT route coverage (route columns will be NaN):")
without_coverage = merged_df[merged_df['IR_HAS'].isna()].head(5)
display(without_coverage)

Sample assets WITH route coverage:


,ASSETNUM,METER_COUNT,MAX_LASTREADING_DATE,ASSET_DESCRIPTION,DEPT_METERS,CLASS_METERS,READING_WITHIN_1YR,HAS_GM,ASSET_DESC,CLASS_ROUTES,DEPT_ROUTES,IR_HAS,UL_HAS,VI_HAS,LU_HAS,MC_HAS,ZD_HAS
0,A100001,83,NaT,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,N,N,"PEOPLE MOVER, OHC-1, WT02 PLATFORM",Conveyor,1AF,Y,N,N,N,N,N
4,A100005,84,2025-12-05 01:01:06.922,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,Y,KNUCKLE ISLAND BEARING PRESS #2,Press,1AF,Y,N,N,N,N,N
5,A100006,84,2024-05-23 12:16:17.124,KNUCKLE ISLAND BEARING PRESS #1,Press,1AF,N,N,KNUCKLE ISLAND BEARING PRESS #1,Press,1AF,Y,N,N,N,N,N
23,A100024,2,2026-01-24 12:00:04.000,"RSAT, ELECTRICAL PANEL, HMI 1, CONVEYOR LOAD",Panel,1AF,Y,Y,"RSAT, ELECTRICAL PANEL, HMI 1, CONVEYOR LOAD",Panel,1AF,Y,N,N,N,N,N
37,A100042,1,2026-01-24 12:00:04.000,SMURF ENGINE SUSPENSION INSTALL ROBOT FANUC MA...,Manipulator,1AF,Y,Y,SMURF ENGINE SUSPENSION INSTALL ROBOT FANUC MA...,Manipulator,1AF,N,N,N,Y,N,N




Sample assets WITHOUT route coverage (route columns will be NaN):


,ASSETNUM,METER_COUNT,MAX_LASTREADING_DATE,ASSET_DESCRIPTION,DEPT_METERS,CLASS_METERS,READING_WITHIN_1YR,HAS_GM,ASSET_DESC,CLASS_ROUTES,DEPT_ROUTES,IR_HAS,UL_HAS,VI_HAS,LU_HAS,MC_HAS,ZD_HAS
1,A100002,91,2026-01-20 20:27:09.260,SMURF REAR INSTALL ROBOT END OF ARM TOOL - ODY...,Head Assembly,1AF,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A100003,84,2024-05-10 17:11:50.072,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - RI...,Head Assembly,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A100004,88,2026-01-20 20:28:24.798,SMURF FRONT INSTALL ROBOT END OF ARM TOOL - OD...,Head Assembly,1AF,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A100007,1,NaT,REAR KNUCKLE AUTO TIGHTEN (PARENT),Power Tool,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A100008,1,NaT,"INLINE BURNISHING MACHINE (PARENT), BURNISHER",Grinder,1AF,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 10: Save Merged Dataset

In [ ]:
# Save as pickle
output_pkl = '/mnt/user-data/outputs/meters-routes-coverage.pkl'
with open(output_pkl, 'wb') as f:
    pickle.dump(merged_df, f)
print(f"✓ Saved: {output_pkl}")

# Save as CSV
output_csv = '/mnt/user-data/outputs/meters-routes-coverage.csv'
merged_df.to_csv(output_csv, index=False)
print(f"✓ Saved: {output_csv}")

print(f"\nMerge complete! Files saved to /mnt/user-data/outputs/")

### Optional: Quick Data Quality Checks

In [ ]:
# Check for any duplicates in ASSETNUM
duplicates = merged_df['ASSETNUM'].duplicated().sum()
print(f"Duplicate ASSETNUMs in merged dataset: {duplicates}")

# Check for null values
print("\nNull value counts:")
print(merged_df.isnull().sum())

In [ ]:
# Summary by department
if 'DEPT_METERS' in merged_df.columns:
    dept_col = 'DEPT_METERS'
else:
    dept_col = 'DEPT'

print(f"\nRecords by Department (using {dept_col}):")
print(merged_df[dept_col].value_counts().head(10))